In [4]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.4 MB/s eta 0:00:00


In [2]:
import mlflow
mlflow.set_tracking_uri("http://98.84.31.179:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2024/12/15 19:15:17 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://dsmp-mlflow-bucket/843005656731549383', creation_time=1734270337551, experiment_id='843005656731549383', last_update_time=1734270337551, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [6]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [7]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [8]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for BOW

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using BOW, fit on training data only
vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_BOW_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2024-12-15 19:22:16,203] A new study created in memory with name: no-name-c4859e2c-830c-44a6-a77c-b124ee74a6a0
[I 2024-12-15 19:22:28,937] Trial 0 finished with value: 0.7245329333151507 and parameters: {'n_estimators': 174, 'learning_rate': 0.04025392741280094, 'max_depth': 7}. Best is trial 0 with value: 0.7245329333151507.
[I 2024-12-15 19:22:35,036] Trial 1 finished with value: 0.5428883131051412 and parameters: {'n_estimators': 135, 'learning_rate': 0.001213130241137026, 'max_depth': 4}. Best is trial 0 with value: 0.7245329333151507.
[I 2024-12-15 19:23:10,999] Trial 2 finished with value: 0.5787535797081685 and parameters: {'n_estimators': 230, 'learning_rate': 0.00010454537273051012, 'max_depth': 8}. Best is trial 0 with value: 0.7245329333151507.
[I 2024-12-15 19:23:12,556] Trial 3 finished with value: 0.5229783171962362 and parameters: {'n_estimators': 67, 'learning_rate': 0.002141632917536649, 'max_depth': 3}. Best is trial 0 with value: 0.7245329333151507.
[I 2024-12-15 

🏃 View run XGBoost_SMOTE_BOW_Trigrams at: http://98.84.31.179:5000/#/experiments/843005656731549383/runs/89b30926599748be806c93b7fabb9b07
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/843005656731549383
